## Train model on saved H,y and dump

In [ ]:
import pandas as pd
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from scipy.optimize import minimize
from scipy.optimize import leastsq

In [ ]:
H = pd.read_csv('output/H_100_g.csv', dtype = float)
y = pd.read_csv('output/y_100_g.csv',header=None, dtype = float )
print len(H), len(y)

z =  y.values 
model = linear_model.LinearRegression()
model.fit(H, z)
p = model.predict(H)

In [ ]:
from sklearn.externals import joblib
joblib.dump(model, 'model/model.pkl')

# PCA & clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

kmeans_model = KMeans(n_clusters=2, random_state=1)
kmeans_model.fit(train)
labels = kmeans_model.labels_
# Create a PCA model.
pca_2 = PCA(2)
# Fit the PCA model on the numeric columns from earlier.
plot_columns = pca_2.fit_transform(train)
# Make a scatter plot of each game, shaded according to cluster assignment.
plt.scatter(x=plot_columns[:,0], y=plot_columns[:,1], c=labels)
# Show the plot.
plt.show()

# Input investigation

In [ ]:
f = train[(train[target]>0)&(train[target]<33)]
print len(f)
f[target].hist(bins = 100)

In [ ]:
train.loc[train[target] == 0, target] = 1e-3
train[target] = np.log(train[target])
train.describe()

In [ ]:
l1 = len(train['Cliente_ID'].unique())
l2 = len(train['Producto_ID'].unique())
l3 = len(train['Ruta_SAK'].unique())
l4 = len(train['Agencia_ID'].unique())
l5 = len(train['Canal_ID'].unique())
l6 = len(train['Semana'].unique())
l7 = len(train['Demanda_uni_equil'].unique())
print l1, l2, l3 ,l4, l5, l6 , l7
print l1*l2*l3*l4*l5
print l1*l2*l3*l4*l5*l6
print len(train)

In [ ]:
demands = pd.DataFrame({ 'count' : data.groupby(by = target)[target].count() }).reset_index()
l = len(data)
demands['p'] = demands['count']/l
print l, demands['p'].sum()
demands = demands.sort_values(by = 'p', ascending = False)

In [ ]:
minmax = pd.DataFrame({ 'delta' : data.groupby(by = train_cols[:-1])[target].max() - data.groupby(by = train_cols[:-1])[target].min() }).reset_index()
minmax = minmax.sort_values(by = 'delta', ascending = False)
minmax

In [ ]:
groupby = get_cols('123456')
train = train.sort_values(by = groupby)
train.head(100)

# Pandas

In [ ]:
train = data[['Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Demanda_uni_equil']].head(1000000)
train = data.head(1000000)
train.loc[train[target] == 0, target] = 1e-3

# Outliers

http://stackoverflow.com/questions/34782063/how-to-use-pandas-filter-with-iqr

In [ ]:
y = pd.Series([3,4,9,10,4,-10,56,345])

print y.sort_values()
print outliers(y)
print outliers_skewed(y)
print y.median(), median_outliers(y)
print y.mean(), mean_outliers(y)

5    -10
0      3
1      4
4      4
2      9
3     10
6     56
7    345
dtype: int64
(-22.875, 48.125)
(-9.9197307942427599, 65.397203832390915)
6.5 6.5
52.625 14.3333333333

print outliers(train[target]), outliers_skewed(train[target])
print train[target].mean(), mean_outliers(train[target])
print train[target].median(), median_outliers(train[target])

groupby = train_cols[:-2]
subf_outliers = pd.DataFrame(data.groupby(by = groupby)[target].agg({'count' : len, 'lower' : outlier_lower, 'upper' : outlier_upper, 'lower_skewed' : outlier_skewed_lower, 'upper_skewed' : outlier_skewed_upper  })).reset_index()
subf_outliers = subf_outliers[subf_outliers['count']>4]
subf_outliers.head()

print outliers(train[target]), outliers_skewed(train[target])
print train[target].mean(), mean_outliers(train[target])
print train[target].median(), median_outliers(train[target])

(-4.0, 12.0) (1.1879883005803242, 32.890134422028403)
7.22456400381 6.06073623938
3.0 4.0


In [ ]:
test1 = data[(data['Cliente_ID'] == 1440936) & (data['Producto_ID'] == 1182) & (data['Ruta_SAK'] == 1027) & (data['Agencia_ID'] == 1245) & (data['Canal_ID'] == 1)]
#test1 = test1[:-1]
print outliers(test1[target]), outliers_skewed(test1[target])
print test1[target].mean(), mean_outliers(test1[target]), mean_outliers_skewed(test1[target])
print test1[target].median(), median_outliers(test1[target]), median_outliers_skewed(test1[target])
test1

In [ ]:
test2 = data[(data['Cliente_ID'] == 4712499) & (data['Producto_ID'] == 30571) & (data['Ruta_SAK'] == 900) & (data['Agencia_ID'] == 2211) & (data['Canal_ID'] == 1)]
print outliers(test2[target]), outliers_skewed(test2[target])
print test2[target].mean(), mean_outliers(test2[target])
print test2[target].median(), median_outliers(test2[target]) 
test2

# Features

In [ ]:
import json
test = pd.read_csv('output/outliers1/12345.csv', nrows = 20, dtype = dtype_dict)
test.head()
a = test.iloc[4].list
lst = pd.Series(json.loads(a))
lst.mean()

In [ ]:
g = train.groupby(by = groupby)[target]
gm = g.mean()
gs = g.size()
gx = g.max()
gn = g.min()
gres = (gm*gs - gx - gn)/(gs-2)

In [ ]:
features = ['12345', '1245', '1234', '124', '1235', '125', '123', '12', '2345', '245', '234', '24', '235', '25',
                          '23', '1345', '145', '134', '14', '135', '15', '13', '1', '2',
                          '345', '45', '4', '5', '3']

In [ ]:
feature_full_list = ['12345', '1234', '1235', '1245', '1345', '2345',
                     '123', '124', '125', '134', '135', '145', '234', '235', '245', '345',
                     '12', '13', '14', '15', '23', '24', '25', '34', '35', '45',
                     '1', '2', '3', '4', '5']

feature_full_list_ = []

for f in feature_full_list:
    feature_full_list_.append(f+'a')

feature_full_list = feature_full_list + feature_full_list_
print feature_full_list

In [ ]:
feature_full_list = ['12345', '1234', '1235', '1245', '1345', '2345', '123', '124', '125', '134', '135', '145', '234', '235',
                     '245', '345', '12', '13', '14', '15', '23', '24', '25', '34', '35', '45', '1', '2', '3', '4', '5',
                     '12345a', '1234a', '1235a', '1245a', '1345a', '2345a', '123a', '124a', '125a', '134a', '135a', '145a',
                     '234a', '235a', '245a', '345a', '12a', '13a', '14a', '15a', '23a', '24a', '25a', '34a', '35a', '45a',
                     '1a', '2a', '3a', '4a', '5a']

feature_full_list = ['134a', '135a', '145a',
                     '234a', '235a', '245a', '345a']

In [ ]:
feature_full_list = ['12345a', '1245a 3a', '1245a', '1234a 5a', '1234a', '1235a 4a', '1235a', 
                           '124a 35a', '124a 3a 5a', '124a', '125a 34a', '125a 3a 4a', '125a',
                           '123a 45a', '123a 4a 5a', '123a',
                           '345a 12a', '12a 45a 3a', '12a 34a 5a', '12a 35a 4a', '12a 3a 4a 5a', '12a',
                           '2345a 1345a', '2345a 145a', '2345a 134a', '2345a 135a',
                           '2345a 14a', '2345a 15a', '2345a 13a', '2345a 1a', '2345a',
                           '245a 1a 3a', '245a 13a',  '245a', '14a 25a 3a', '24a 1a 3a 5a', '15a 24a 3a', '13a 24a 5a',
                           '234a 1a 5a', '235a 14a', '24a 35a 1a', '25a 1a 3a 4a', '235a 1a 4a', '134a 25a', '234a 15a',
                           '135a 24a', '13a 25a 4a', '25a 34a 1a', '14a 23a 5a', '145a 23a', '15a 23a 4a', '234a',
                           '24a', '23a 1a 4a 5a', '145a 2a 3a', '235a', '14a 2a 3a 5a', '134a 2a 5a', '14a 35a 2a',
                           '1345a 2a', '23a 45a 1a', '15a 2a 3a 4a', '135a 2a 4a', '25a', '15a 34a 2a', '45a 1a 2a 3a',
                           '1a 2a 3a 4a 5a', '13a 2a 4a 5a', '34a 1a 2a 5a', '13a 45a 2a', '345a 1a 2a', '35a 1a 2a 4a',
                           '1345a', '145a 3a', '145a', '14a 3a 5a', '134a 5a', '14a 35a',
                           '134a', '14a', '23a', '135a 4a', '135a', '15a 34a', '15a 3a 4a', '15a', '45a 1a 3a',
                           '13a 45a', '345a 1a', '1a 3a 4a 5a', '13a 4a 5a', '34a 1a 5a', '13a', '35a 1a 4a',
                           '1a', '2a', '345a', '45a', '35a', '34a', '4a', '5a', '3a']

In [ ]:
for i in range(0, len(feature_full_list)):
    feature_full_list[i] = feature_full_list[i].replace('a', '')
#feature_full_list  

In [ ]:
for i in range(1, len(features_trained_mean_sorted)):
    features_trained_mean_sorted[i] = features_trained_mean_sorted[i].replace('a', '')

In [ ]:
features_trained_mean_sorted = ['12345a', '1245a 3a', '1245a', '1234a 5a', '1234a', '1235a 4a', '1235a', 
                           '124a 35a', '124a 3a 5a', '124a', '125a 34a', '125a 3a 4a', '125a',
                           '123a 45a', '123a 4a 5a', '123a',
                           '345a 12a', '12a 45a 3a', '12a 34a 5a', '12a 35a 4a', '12a 3a 4a 5a', '12a',
                           '2345a 1345a', '2345a 145a', '2345a 134a', '2345a 135a',
                           '2345a 14a', '2345a 15a', '2345a 13a', '2345a 1a', '2345a',
                           '245a 1a 3a', '245a 13a',  '245a', '14a 25a 3a', '24a 1a 3a 5a', '15a 24a 3a', '13a 24a 5a',
                           '234a 1a 5a', '235a 14a', '24a 35a 1a', '25a 1a 3a 4a', '235a 1a 4a', '134a 25a', '234a 15a',
                           '135a 24a', '13a 25a 4a', '25a 34a 1a', '14a 23a 5a', '145a 23a', '15a 23a 4a', '234a',
                           '24a', '23a 1a 4a 5a', '145a 2a 3a', '235a', '14a 2a 3a 5a', '134a 2a 5a', '14a 35a 2a',
                           '1345a 2a', '23a 45a 1a', '15a 2a 3a 4a', '135a 2a 4a', '25a', '15a 34a 2a', '45a 1a 2a 3a',
                           '1a 2a 3a 4a 5a', '13a 2a 4a 5a', '34a 1a 2a 5a', '13a 45a 2a', '345a 1a 2a', '35a 1a 2a 4a',
                           '1345a', '145a 3a', '145a', '14a 3a 5a', '134a 5a', '14a 35a',
                           '134a', '14a', '23a', '135a 4a', '135a', '15a 34a', '15a 3a 4a', '15a', '45a 1a 3a',
                           '13a 45a', '345a 1a', '1a 3a 4a 5a', '13a 4a 5a', '34a 1a 5a', '13a', '35a 1a 4a',
                           '1a', '2a', '345a', '45a', '35a', '34a', '4a', '5a', '3a']

features_trained_mean = {'13a 45a': [-0.8284296431414946, 0.8182684185131366, 0.2964000435152635, 299.91773414618734], '23a 1a 4a 5a': [-0.7927464034040126, 0.8150182648019166, 0.8026696821910526, 0.0705075712029412, -0.5784662109342337, 228.86776784817377], '235a': [-3.1938895972416503e-12, 1.0000000000004532, 232.08179350189417], '14a': [-2.2657431486550195e-12, 1.00000000000032, 283.17951211921786], '2345a 1a': [-1.3930592672202602, 0.8745262278207452, 0.3182963793963406, 131.97828986051962], '235a 1a 4a': [-0.9286748013984285, 0.835075706162021, 0.6279369298496867, -0.3344685742649066, 195.03878560648542], '12a': [-7.540634783254063e-12, 1.0000000000010474, 70.1115592380727], '345a 1a 2a': [-3.451112216191265, 0.15303825780711433, 0.6739321342659998, 0.6507210249277119, 253.9668470130267], '135a 24a': [-1.8752651860006528, 0.5026823240269876, 0.7568856146074984, 196.2975471057333], '14a 3a 5a': [-0.01741173852452782, 0.9932551090843875, -0.08571987002958711, 0.09487483558989215, 283.0833814961033], '245a 13a': [-1.3989452187857987, 0.8718809884047755, 0.32175633255315933, 131.62349389379642], '24a 35a 1a': [-0.7956880815354959, 0.8555750274535269, -0.3924056417092947, 0.6469670987712107, 191.44897721350364], '134a': [-4.4586556668946287e-13, 1.0000000000000682, 283.1735276257144], '134a 2a 5a': [-2.1479189815709008, 0.884780667163834, 0.68137720267706, -0.2688501042181243, 233.37869633393854], '24a': [-1.1732836924238654e-12, 1.0000000000001739, 227.5657008779384], '135a': [3.232969447708456e-13, 0.9999999999999616, 292.60549165182766], '12a 45a 3a': [-0.0015499422521241257, 0.9864040190409419, 0.10178325960573156, -0.08797274082110035, 69.6699359459314], '123a 45a': [-0.2472861028952007, 0.9862120564168557, 0.048016459347550826, 68.54359782567951], '14a 2a 3a 5a': [-1.9032306891633501, 0.890963561006628, 0.6917794036385623, -0.32200707778943005, 0.0027029432466311467, 232.34483292831828], '2345a 15a': [-1.3054680171642206, 0.861919731256727, 0.31877878791710373, 131.60309930149302], '15a 34a 2a': [-2.528416104724406, 0.8651052610607949, -0.19377874347388924, 0.6786483879425986, 242.2197238359209], '25a 34a 1a': [-1.1984533878433625, 0.8305798035227088, -0.33314423277604377, 0.6684503411523254, 199.81813929090382], '1234a': [1.0088818669373723e-11, 0.9999999999986012, 61.11900488418285], '235a 14a': [-2.0585875978196713, 0.7341806040812046, 0.5507622100090804, 191.2959522579031], '1a': [-3.2738256550146616e-12, 1.0000000000005604, 306.9578344073995], '345a 12a': [-0.25716226833705136, 0.049973209456649674, 0.9856223321153558, 69.89399209048801], '15a 34a': [-0.17664066642397458, 0.9764431155971606, 0.04800689499600056, 292.8626159851131], '24a 1a 3a 5a': [-0.038308056689879244, 0.8592491508044457, 0.6694751540297097, -0.5845205032002442, 0.06109867152154694, 187.66800959004865], '12a 34a 5a': [-0.10064939020031716, 0.9953837057673393, 0.037757063275500644, -0.019209216890029572, 70.0664022095164], '5a': [-4.445421808441097e-11, 1.0000000000061784, 394.3222130822432], '14a 25a 3a': [-0.10483711905492754, 0.7225824475217618, 0.8120442907157904, -0.5201155346549744, 185.015090017214], '1345a': [-8.100187187665142e-13, 1.000000000000114, 280.7739496817556], '13a': [-3.2027713814386516e-12, 1.0000000000005507, 304.92486991292816], '13a 25a 4a': [-1.0658249457274396, 0.662984736877662, 0.8235712310176109, -0.33902804161952343, 198.98462495841892], '25a': [-1.4157564010019996e-12, 1.0000000000001947, 241.7757918790244], '134a 5a': [-0.09090859271584595, 0.9911760912913219, 0.021407172234062865, 283.15187770155705], '124a 3a 5a': [-0.01275365630222236, 0.9982256390671557, -0.06611284218415867, 0.06965252159095633, 61.07225042545344], '45a': [5.755396159656812e-13, 0.9999999999999223, 353.43603195044005], '245a': [1.4477308241112041e-13, 0.999999999999981, 143.7044063776906], '2345a 14a': [-1.3379848915619528, 0.8475714219714325, 0.33762797426302127, 129.6239192709875], '34a': [-2.957012412707627e-11, 1.0000000000040734, 382.8810474637211], '15a 3a 4a': [-0.03069136576293996, 0.9906858729941826, -0.055181312844572235, 0.06874363610788015, 292.889827716837], '234a 15a': [-1.8651354399331899, 0.7590592354931011, 0.49910657755122917, 195.8221237880713], '15a 24a 3a': [-0.021031372427986028, 0.6426782418369897, 0.8252967672124777, -0.46506391677916836, 187.78417457623038], '13a 4a 5a': [-0.4722223468323614, 0.9504983945278211, 0.2670909607187001, -0.15222591648615053, 303.10577256367435], '2345a': [4.662936703425657e-13, 0.9999999999999362, 143.70033786172652], '35a 1a 4a': [-0.7149036083304692, 0.08357866891235369, 0.9104815122395851, 0.10489438661915912, 305.4029589721721], '125a 3a 4a': [-0.006380954556208351, 0.9993084294164557, -0.013748747621321784, 0.015323548605045248, 61.208693377382225], '2a': [-2.3137047833188262e-12, 1.0000000000001685, 337.20701743250805], '1a 3a 4a 5a': [-0.20977823877367374, 0.9549905741477905, -0.5043729231209124, 0.4273570640178398, 0.1510620875448467, 303.0702053332763], '1235a 4a': [-0.02416592371998849, 0.9991548114374647, 0.0041901549449033714, 61.174716931198866], '1a 2a 3a 4a 5a': [-2.168062123610313, 0.8506564237972264, 0.7207907070461838, -0.7334078944223993, 0.4194352193088483, 0.042621456281953365, 247.70783667909143], '234a 1a 5a': [-0.5157798452380584, 0.8438640485741346, 0.6478107313716016, -0.42028225785487133, 190.46195084281794], '2345a 1345a': [-1.3277944775346233, 0.844865043443719, 0.338923829744365, 129.4552355860161], '15a 23a 4a': [-0.9933435536722621, 0.8170465214610618, 0.7670009838281024, -0.44655221101955817, 221.0664036461994], '35a 1a 2a 4a': [-2.865402986683198, -0.0961070212312538, 0.8018980689528429, 0.6828896346711583, 0.007938820898199618, 254.88280908735285], '12a 35a 4a': [-0.06598439106904053, 0.9969118383719169, 0.02725095343414071, -0.015029452958991704, 70.08972861306184], '145a 3a': [0.00021052120571507515, 1.0000173244490789, -4.646409066670258e-05, 280.7799340777297], '12a 3a 4a 5a': [-0.02365289673223625, 0.9970998582533731, -0.08751034304083591, 0.021608570290960017, 0.07207586939810778, 70.03963711731541], '125a 34a': [-0.10589309805211222, 0.9959267697284818, 0.018730598937132883, 61.187949576411754], '2345a 134a': [-1.33801599291973, 0.8475618467623598, 0.33764185441807565, 129.62241279114502], '123a': [2.6350477355663315e-11, 0.999999999996356, 68.74449359871974], '1245a 3a': [0.0001830362199841673, 1.0000051944680164, -3.052972975600567e-05, 50.59545661129897], '345a 1a': [-0.8531277464286005, 0.3079534640958719, 0.8101336268316688, 301.54653125909687], '245a 1a 3a': [-0.004588594661172785, 0.8987378152211781, 0.4488316181136887, -0.3469342954000564, 127.17396226363084], '1345a 2a': [-3.0696431975717475, 0.7923336794936106, 0.6325560710787547, 234.3777039869044], '14a 35a': [-0.08227327279340191, 0.989489311200762, 0.021898680206071053, 283.1561622231484], '1234a 5a': [-0.06955152257852859, 0.9977257990563402, 0.011901290210482843, 61.10985675173245], '34a 1a 5a': [-0.42266829348379176, 0.3426180406458091, 0.9348831525196621, -0.21899686082627656, 304.06616901384916], '13a 45a 2a': [-3.4305989325773796, 0.6845703569494709, 0.14047562994788326, 0.6498060499549851, 252.43817146054846], '345a': [4.206412995699793e-12, 0.99999999999942, 353.4337836946128], '1245a': [3.495070899361963e-11, 0.9999999999951613, 50.59545666701653], '14a 23a 5a': [-0.6040130997573252, 0.8373605339993002, 0.7906594236731521, -0.5444144834265648, 210.86066398311138], '15a 2a 3a 4a': [-1.988146135419524, 0.8786745590545513, 0.708172851854859, -0.40480508121690617, 0.0931502126354819, 239.44924863943203], '124a 35a': [-0.05966399197829908, 0.9976360132807378, 0.010622477051254674, 61.11776393054111], '3a': [-4.4508574603696616e-10, 1.0000000000617049, 401.94918626016425], '124a': [1.2424727913185052e-11, 0.9999999999982786, 61.12591069828023], '123a 4a 5a': [-0.06942639376593274, 0.9977709713210324, -0.007077746158545087, 0.018916544192367518, 68.73216633947301], '45a 1a 3a': [-0.008969835004877247, 0.45588053993788513, 0.8502144320253321, -0.30485339738506934, 298.9805273851467], '45a 1a 2a 3a': [-1.9885270676463849, 0.441366492339137, 0.7419962225111539, 0.7200604994617634, -0.6281779446185127, 243.6081309923466], '34a 1a 2a 5a': [-2.4684291716804525, 0.24867322975585177, 0.8340102747483078, 0.7012861694803789, -0.4422979592097873, 251.3417502501992], '12345a': [2.0044410575792426e-11, 0.9999999999972238, 50.58855085291899], '13a 2a 4a 5a': [-2.504523012169222, 0.847071585788149, 0.702610840457993, 0.19098737503626997, -0.3940020977573569, 250.10825105827047], '23a': [7.529976642217662e-12, 0.9999999999990754, 288.4171877426784], '15a': [-1.816324868286756e-12, 1.0000000000002587, 292.9721306198166], '2345a 13a': [-1.398886401528773, 0.8718860867035003, 0.32174309296632403, 131.6205010271264], '2345a 135a': [-1.3043951042114665, 0.8615164741200534, 0.31903353602926193, 131.57320611112812], '25a 1a 3a 4a': [-0.18277171268867676, 0.8559158620117006, 0.729900099633813, -0.7207482070076745, 0.16023089488516107, 191.85321891007777], '23a 45a 1a': [-2.3943806267656997, 0.6974524671457447, 0.011789827985464754, 0.6221798675772254, 239.2181344927566], '14a 35a 2a': [-2.3787917628466007, 0.8872222846904855, -0.2284910286651412, 0.670533148247742, 234.1530293957683], '145a': [3.517186542012496e-13, 0.9999999999999534, 280.7799341752589], '145a 2a 3a': [-1.899109176845717, 0.8927654744386512, 0.6904637205555867, -0.3203608508499596, 230.13545069411475], '35a': [-3.728395370217186e-11, 1.0000000000051967, 381.3379158569302], '125a': [9.497291841853439e-12, 0.9999999999986853, 61.21306362370972], '135a 4a': [-0.1000788942498172, 0.9884739671320253, 0.02537861887830712, 292.57577755035135], '13a 24a 5a': [-0.5221142945374604, 0.6520967108332683, 0.839332572662125, -0.41915996810654366, 190.30516402495016], '1235a': [-6.198597191087174e-12, 1.0000000000008589, 61.1758889054093], '145a 23a': [-2.44085658163094, 0.6821341132270663, 0.6557210957994926, 220.51255995081863], '135a 2a 4a': [-2.4229044904691284, 0.8647406667365751, 0.6782764709235238, -0.207646797084241, 241.7137573615264], '2345a 145a': [-1.327764013795922, 0.8448747272798076, 0.33890992921939606, 129.4567511020937], '234a': [-1.0933476346508542e-12, 1.0000000000001639, 226.54949558746802], '134a 25a': [-2.175097756625889, 0.5783331039349435, 0.7227366568810929, 195.80635150613213], '4a': [-2.6508573114369938e-11, 1.0000000000036993, 390.4381940141035]}

## Median

In [ ]:
sorted_medi_full_train = ['12345', '1245', '1234', '124', '1235', '125', '123', '12', '2345', '245', '234', '24',
                          '235', '25',
                          '23', '1345', '145', '134', '14', '135',
                          '15', '13', '1', '2', '345', '45', '35', '34', '4', '5', '3']
                          
sorted_list_val15 = ['12345', '1245', '1234', '124', '125', '1235', '123', '12', '2345', '245', '235', '234', '24', '25',
                          '23', '145', '1345', '14', '134', '135', '15', '13', '1', '2',
                          '345', '45', '35', '34', '5', '4', '3']
                          
feature_full_evaluation_list = {'12345':57.6363712554,
'1234':68.3218248675,
'1235':68.5490921168,
'1245':57.6435429549,
'1345':318.386493869,
'2345':164.001794987,
'123':76.1343145616,
'124':68.3289965671,
'125':68.588916443,
'134':320.901127533,
'135':329.292927178,
'145':318.392726682,
'234':249.17814807,
'235':251.826838458,
'245':164.006071147,
'345':384.71309467,
'12':77.6456723161,
'13':343.018141758,
'14':320.907360346,
'15':329.696049825,
'23':309.898539446,
'24':250.086799511,
'25':261.601460035,
'34':409.670563459,
'35':409.256696339,
'45':384.714720502,
'1':345.066569886,
'2':360.636458704,
'3':427.034442721,
'4':416.543269067,
'5':420.547032343}                         

## Mean

In [ ]:
sorted_mean_full_train = ['12345a', '1245a', '1234a', '124a', '1235a', '125a', '123a', '12a', '2345a', '245a', '234a', '24a',
                          '235a', '25a',
                          '1345a', '145a', '134a', '14a', '23a', '135a',
                          '15a', '13a', '1a', '2a', '345a', '45a', '35a', '34a', '4a', '5a', '3a']

In [ ]:
feature_list = ['12345a', '1245a', '1234a', '124a', '1235a', '125a', '123a', '12a', '2345a', '245a', '234a', '24a',
                '235a', '25a',
                '1345a', '145a', '134a', '14a', '23a', '135a',
                '15a', '13a', '1a', '2a', '345a', '45a', '35a', '34a', '4a', '5a', '3a']

In [ ]:
features = ['12345a', '1245a', '1234a', '124a', '1235a', '125a', '123a', '12a', '2345a', '245a', '234a', '24a',
                          '235a', '25a',
                          '1345a', '145a', '134a', '14a', '23a', '135a',
                          '15a', '13a', '1a', '2a', '345a', '45a', '35a', '34a', '4a', '5a', '3a']

In [ ]:
feature_full_list = ['12345a',
                     '1234a 5a', '1235a 4a', '1245a 3a', '1345a 2a', '2345a 1a',
                     '123a 45a', '124a 35a', '125a 34a', '134a 25a', '135a 24a', '145a 23a',
                     '234a 15a', '235a 14a', '245a 13a', '345a 12a',
                     '123a 4a 5a', '124a 3a 5a', '125a 3a 4a', '134a 2a 5a', '135a 2a 4a', '145a 2a 3a',
                     '234a 1a 5a', '235a 1a 4a', '245a 1a 3a', '345a 1a 2a',
                     '12a 34a 5a', '12a 35a 4a', '12a 45a 3a', '13a 24a 5a', '13a 25a 4a', '13a 45a 2a',
                     '14a 23a 5a', '14a 25a 3a', '14a 35a 2a', '15a 23a 4a', '15a 24a 3a', '15a 34a 2a',
                     '23a 45a 1a', '24a 35a 1a', '25a 34a 1a', 
                     '12a 3a 4a 5a', '13a 2a 4a 5a', '14a 2a 3a 5a', '15a 2a 3a 4a',
                     '23a 1a 4a 5a', '24a 1a 3a 5a', '25a 1a 3a 4a', '34a 1a 2a 5a',
                     '35a 1a 2a 4a', '45a 1a 2a 3a',
                     '1a 2a 3a 4a 5a',
                     
                     #non existing products
                     '1345a',
                     '134a 5a', '135a 4a', '145a 3a', '345a 1a', 
                     '13a 45a', '14a 35a', '15a 34a',
                     '13a 4a 5a', '14a 3a 5a', '15a 3a 4a', '34a 1a 5a', '35a 1a 4a', '45a 1a 3a',
                     '1a 3a 4a 5a',
                     
                     '1234a', '1235a', '1245a', '1345a', '2345a', '123a', '124a', '125a', '134a', '135a', '145a',
                     '234a', '235a', '245a', '345a', '12a', '13a', '14a', '15a', '23a', '24a', '25a', '34a', '35a', '45a',
                     '1a', '2a', '3a', '4a', '5a'
                    ]

In [ ]:
features_trained = {'13a 45a': [-0.82842964314149459, 0.81826841851313659, 0.2964000435152635, 299.91773414618734], '23a 1a 4a 5a': [-0.79274640340401259, 0.81501826480191664, 0.80266968219105261, 0.070507571202941197, -0.57846621093423367, 228.86776784817377], '34a 1a 5a': [-0.42266829348379176, 0.34261804064580909, 0.93488315251966214, -0.21899686082627656, 304.06616901384916], '15a 23a 4a': [-0.99334355367226213, 0.81704652146106183, 0.76700098382810245, -0.44655221101955817, 221.0664036461994], '2345a 1a': [-1.3930592672202602, 0.87452622782074518, 0.31829637939634059, 131.97828986051962], '235a 1a 4a': [-0.92867480139842851, 0.83507570616202098, 0.62793692984968674, -0.33446857426490662, 195.03878560648542], '345a 1a 2a': [-3.4511122161912651, 0.15303825780711433, 0.67393213426599985, 0.65072102492771189, 253.96684701302669], '135a 24a': [-1.8752651860006528, 0.50268232402698765, 0.75688561460749837, 196.29754710573329], '1a 3a 4a 5a': [-0.20977823877367374, 0.95499057414779054, -0.50437292312091242, 0.42735706401783979, 0.15106208754484671, 303.07020533327631], '12a 3a 4a 5a': [-0.02365289673223625, 0.99709985825337311, -0.087510343040835908, 0.021608570290960017, 0.072075869398107784, 70.039637117315408], '125a 34a': [-0.10589309805211222, 0.99592676972848182, 0.018730598937132883, 61.187949576411754], '245a 13a': [-1.3989452187857987, 0.87188098840477546, 0.32175633255315933, 131.62349389379642], '24a 35a 1a': [-0.79568808153549586, 0.85557502745352687, -0.39240564170929471, 0.64696709877121072, 191.44897721350364], '1245a 3a': [0.0001830362199841673, 1.0000051944680164, -3.052972975600567e-05, 50.595456611298971], '134a 2a 5a': [-2.1479189815709008, 0.88478066716383397, 0.68137720267706003, -0.26885010421812428, 233.37869633393854], '145a 3a': [0.00021052120571507515, 1.0000173244490789, -4.6464090666702581e-05, 280.77993407772971], '345a 1a': [-0.85312774642860045, 0.30795346409587188, 0.81013362683166878, 301.54653125909687], '12a 45a 3a': [-0.0015499422521241257, 0.98640401904094188, 0.10178325960573156, -0.087972740821100348, 69.6699359459314], '123a 45a': [-0.2472861028952007, 0.98621205641685572, 0.048016459347550826, 68.543597825679512], '245a 1a 3a': [-0.0045885946611727846, 0.89873781522117813, 0.44883161811368871, -0.34693429540005638, 127.17396226363084], '1345a 2a': [-3.0696431975717475, 0.79233367949361055, 0.63255607107875467, 234.37770398690441], '23a 45a 1a': [-2.3943806267656997, 0.69745246714574471, 0.011789827985464754, 0.62217986757722543, 239.2181344927566], '15a 34a 2a': [-2.528416104724406, 0.86510526106079488, -0.19377874347388924, 0.67864838794259863, 242.21972383592089], '25a 34a 1a': [-1.1984533878433625, 0.83057980352270877, -0.33314423277604377, 0.66845034115232538, 199.81813929090382], '1234a 5a': [-0.069551522578528591, 0.99772579905634018, 0.011901290210482843, 61.109856751732451], '235a 14a': [-2.0585875978196713, 0.73418060408120456, 0.55076221000908043, 191.29595225790311], '13a 45a 2a': [-3.4305989325773796, 0.68457035694947088, 0.14047562994788326, 0.64980604995498514, 252.43817146054846], '35a 1a 2a 4a': [-2.8654029866831978, -0.096107021231253795, 0.80189806895284288, 0.68288963467115826, 0.0079388208981996181, 254.88280908735285], '345a 12a': [-0.25716226833705136, 0.049973209456649674, 0.98562233211535577, 69.893992090488013], '15a 34a': [-0.17664066642397458, 0.97644311559716057, 0.048006894996000561, 292.86261598511311], '14a 23a 5a': [-0.6040130997573252, 0.83736053399930022, 0.79065942367315212, -0.54441448342656484, 210.86066398311138], '15a 2a 3a 4a': [-1.988146135419524, 0.87867455905455127, 0.70817285185485901, -0.40480508121690617, 0.093150212635481899, 239.44924863943203], '12a 34a 5a': [-0.10064939020031716, 0.99538370576733926, 0.037757063275500644, -0.019209216890029572, 70.066402209516397], '124a 35a': [-0.05966399197829908, 0.99763601328073781, 0.010622477051254674, 61.117763930541109], '14a 25a 3a': [-0.10483711905492754, 0.72258244752176182, 0.81204429071579043, -0.5201155346549744, 185.01509001721399], '1345a': [-8.1001871876651421e-13, 1.0000000000001139, 280.7739496817556], '123a 4a 5a': [-0.069426393765932737, 0.99777097132103243, -0.007077746158545087, 0.018916544192367518, 68.732166339473011], '14a 3a 5a': [-0.01741173852452782, 0.99325510908438752, -0.085719870029587114, 0.094874835589892154, 283.0833814961033], '14a 2a 3a 5a': [-1.9032306891633501, 0.89096356100662799, 0.69177940363856227, -0.32200707778943005, 0.0027029432466311467, 232.34483292831828], '13a 25a 4a': [-1.0658249457274396, 0.66298473687766202, 0.82357123101761087, -0.33902804161952343, 198.98462495841892], '45a 1a 2a 3a': [-1.9885270676463849, 0.441366492339137, 0.74199622251115394, 0.72006049946176343, -0.62817794461851273, 243.60813099234659], '34a 1a 2a 5a': [-2.4684291716804525, 0.24867322975585177, 0.8340102747483078, 0.70128616948037892, -0.44229795920978732, 251.3417502501992], '12345a': [2.0044410575792426e-11, 0.99999999999722378, 50.588550852918992], '13a 2a 4a 5a': [-2.5045230121692219, 0.84707158578814901, 0.70261084045799305, 0.19098737503626997, -0.39400209775735689, 250.10825105827047], '134a 5a': [-0.090908592715845948, 0.99117609129132189, 0.021407172234062865, 283.15187770155705], '25a 1a 3a 4a': [-0.18277171268867676, 0.85591586201170056, 0.72990009963381297, -0.7207482070076745, 0.16023089488516107, 191.85321891007777], '124a 3a 5a': [-0.01275365630222236, 0.99822563906715567, -0.066112842184158666, 0.069652521590956334, 61.07225042545344], '14a 35a 2a': [-2.3787917628466007, 0.88722228469048547, -0.2284910286651412, 0.67053314824774202, 234.1530293957683], '145a 2a 3a': [-1.8991091768457169, 0.89276547443865117, 0.69046372055558669, -0.32036085084995958, 230.13545069411475], '14a 35a': [-0.082273272793401908, 0.989489311200762, 0.021898680206071053, 283.15616222314839], '15a 3a 4a': [-0.030691365762939959, 0.99068587299418265, -0.055181312844572235, 0.068743636107880146, 292.88982771683698], '135a 4a': [-0.1000788942498172, 0.98847396713202529, 0.02537861887830712, 292.57577755035135], '13a 24a 5a': [-0.52211429453746039, 0.65209671083326826, 0.83933257266212502, -0.41915996810654366, 190.30516402495016], '234a 15a': [-1.8651354399331899, 0.75905923549310106, 0.49910657755122917, 195.82212378807131], '24a 1a 3a 5a': [-0.038308056689879244, 0.85924915080444575, 0.6694751540297097, -0.58452050320024418, 0.061098671521546938, 187.66800959004865], '145a 23a': [-2.44085658163094, 0.6821341132270663, 0.65572109579949256, 220.51255995081863], '135a 2a 4a': [-2.4229044904691284, 0.86474066673657513, 0.67827647092352383, -0.207646797084241, 241.7137573615264], '13a 4a 5a': [-0.47222234683236142, 0.95049839452782114, 0.26709096071870009, -0.15222591648615053, 303.10577256367435], '35a 1a 4a': [-0.71490360833046918, 0.083578668912353687, 0.91048151223958507, 0.10489438661915912, 305.40295897217209], '125a 3a 4a': [-0.0063809545562083514, 0.9993084294164557, -0.013748747621321784, 0.015323548605045248, 61.208693377382225], '234a 1a 5a': [-0.51577984523805842, 0.84386404857413455, 0.64781073137160161, -0.42028225785487133, 190.46195084281794], '15a 24a 3a': [-0.021031372427986028, 0.64267824183698974, 0.82529676721247769, -0.46506391677916836, 187.78417457623038], '1235a 4a': [-0.024165923719988491, 0.99915481143746465, 0.0041901549449033714, 61.174716931198866], '1a 2a 3a 4a 5a': [-2.168062123610313, 0.85065642379722639, 0.72079070704618375, -0.73340789442239929, 0.41943521930884831, 0.042621456281953365, 247.70783667909143], '134a 25a': [-2.1750977566258891, 0.57833310393494353, 0.7227366568810929, 195.80635150613213], '45a 1a 3a': [-0.0089698350048772468, 0.45588053993788513, 0.85021443202533209, -0.30485339738506934, 298.98052738514667], '12a 35a 4a': [-0.065984391069040527, 0.99691183837191688, 0.027250953434140711, -0.015029452958991704, 70.089728613061837]}
features_trained.update( {'12a': [-7.5406347832540632e-12, 1.0000000000010474, 70.111559238072701], '13a': [-3.2027713814386516e-12, 1.0000000000005507, 304.92486991292816], '1235a': [-6.198597191087174e-12, 1.0000000000008589, 61.175888905409302], '24a': [-1.1732836924238654e-12, 1.0000000000001739, 227.56570087793841], '2345a': [4.6629367034256575e-13, 0.99999999999993616, 143.70033786172652], '25a': [-1.4157564010019996e-12, 1.0000000000001947, 241.77579187902441], '23a': [7.5299766422176617e-12, 0.99999999999907541, 288.41718774267838], '45a': [5.7553961596568115e-13, 0.99999999999992228, 353.43603195044005], '15a': [-1.8163248682867561e-12, 1.0000000000002587, 292.9721306198166], '123a': [2.6350477355663315e-11, 0.99999999999635603, 68.744493598719743], '14a': [-2.2657431486550195e-12, 1.00000000000032, 283.17951211921786], '134a': [-4.4586556668946287e-13, 1.0000000000000682, 283.17352762571443], '135a': [3.2329694477084558e-13, 0.99999999999996159, 292.60549165182766], '245a': [1.4477308241112041e-13, 0.99999999999998102, 143.70440637769059], '34a': [-2.9570124127076269e-11, 1.0000000000040734, 382.88104746372107], '124a': [1.2424727913185052e-11, 0.9999999999982786, 61.125910698280229], '145a': [3.5171865420124959e-13, 0.99999999999995337, 280.77993417525892], '35a': [-3.7283953702171857e-11, 1.0000000000051967, 381.33791585693018], '125a': [9.4972918418534391e-12, 0.99999999999868527, 61.213063623709722], '1234a': [1.0088818669373723e-11, 0.99999999999860123, 61.11900488418285], '235a': [-3.1938895972416503e-12, 1.0000000000004532, 232.08179350189417], '1a': [-3.2738256550146616e-12, 1.0000000000005604, 306.9578344073995], '345a': [4.2064129956997931e-12, 0.99999999999942002, 353.43378369461283], '234a': [-1.0933476346508542e-12, 1.0000000000001639, 226.54949558746802], '2a': [-2.3137047833188262e-12, 1.0000000000001685, 337.20701743250805], '1245a': [3.4950708993619628e-11, 0.99999999999516131, 50.595456667016528], '3a': [-4.4508574603696616e-10, 1.0000000000617049, 401.94918626016425], '4a': [-2.6508573114369938e-11, 1.0000000000036993, 390.43819401410349], '5a': [-4.4454218084410968e-11, 1.0000000000061784, 394.3222130822432], '1345a': [-8.1001871876651421e-13, 1.0000000000001139, 280.7739496817556]} )

features_trained.update({'2345a 1345a': [-1.3277944775346233, 0.84486504344371904, 0.33892382974436502, 129.4552355860161], '2345a 134a': [-1.3380159929197299, 0.84756184676235979, 0.33764185441807565, 129.62241279114502], '2345a 135a': [-1.3043951042114665, 0.86151647412005339, 0.31903353602926193, 131.57320611112812], '2345a 13a': [-1.398886401528773, 0.87188608670350032, 0.32174309296632403, 131.62050102712641], '2345a 14a': [-1.3379848915619528, 0.84757142197143254, 0.33762797426302127, 129.6239192709875], '2345a 15a': [-1.3054680171642206, 0.86191973125672705, 0.31877878791710373, 131.60309930149302], '2345a 145a': [-1.3277640137959219, 0.84487472727980761, 0.33890992921939606, 129.4567511020937]} )